In [1]:
import cudaq
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

In [2]:
df = pd.read_csv("../data/noalpha.csv")
df.head()
df.columns

Index(['Unnamed: 0', 'ID', 'RAdeg', 'DEdeg', 'e_RAdeg', 'e_DEdeg', 'RApeak',
       'DEpeak', 'Sint', 'e_Sint', 'Speak', 'e_Speak', 'rmspeak', 'e_rmspeak',
       'thetamaj', 'thetamin', 'PA'],
      dtype='object')

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("nvidia")
device

nvidia


device(type='cuda')

In [4]:
# Define QuantumFunction
class QuantumFunction(torch.autograd.Function):
    def __init__(self, qubit_count: int, hamiltonian: cudaq.SpinOperator):
        @cudaq.kernel
        def kernel(qubit_count: int, thetas: np.ndarray):
            qubits = cudaq.qvector(qubit_count)
            cudaq.ry(thetas[0], qubits[0])
            cudaq.rx(thetas[1], qubits[0])
        self.kernel = kernel
        self.qubit_count = qubit_count
        self.hamiltonian = hamiltonian

    def run(self, theta_vals: torch.Tensor) -> torch.Tensor:
        theta_vals_np = theta_vals.detach().cpu().numpy()
        qubit_count = [self.qubit_count] * theta_vals_np.shape[0]
        results = cudaq.observe(self.kernel, self.hamiltonian, qubit_count, theta_vals_np)
        return torch.tensor([r.expectation() for r in results], device=device)

    @staticmethod
    def forward(ctx, thetas, quantum_circuit, shift):
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit
        ctx.save_for_backward(thetas)
        return ctx.quantum_circuit.run(thetas).unsqueeze(1)

    @staticmethod
    def backward(ctx, grad_output):
        thetas, = ctx.saved_tensors
        gradients = torch.zeros_like(thetas)
        for i in range(thetas.shape[1]):
            thetas_plus = thetas.clone(); thetas_plus[:, i] += ctx.shift
            thetas_minus = thetas.clone(); thetas_minus[:, i] -= ctx.shift
            exp_plus = ctx.quantum_circuit.run(thetas_plus)
            exp_minus = ctx.quantum_circuit.run(thetas_minus)
            gradients[:, i] = (exp_plus - exp_minus) / (2 * ctx.shift)
        return gradients * grad_output, None, None

In [5]:
# Define Ry and Rx since cudaq.ry and cudaq.rx aren't directly callable in kernel
@cudaq.kernel
def ry(theta: float, qubit: int):
    cudaq.ry(theta, qubit)

@cudaq.kernel
def rx(theta: float, qubit: int):
    cudaq.rx(theta, qubit)

In [6]:
# Quantum Layer
class QuantumLayer(nn.Module):
    def __init__(self, qubit_count=2, shift=torch.tensor(torch.pi/2)):
        super().__init__()
        hamiltonian = cudaq.spin.z(0)
        self.qfunc = QuantumFunction(qubit_count, hamiltonian)
        self.shift = shift

    def forward(self, x):
        return QuantumFunction.apply(x, self.qfunc, self.shift)

In [7]:
# HQNN Model for multi-target regression
class HybridQNNMulti(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout1 = nn.Dropout(0.25)
        self.fc3 = nn.Linear(128, 64)
        self.quantum = QuantumLayer()
        self.fc_out = nn.Linear(1, 2)  # predict both outputs

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc3(x))
        x = self.quantum(x)
        return self.fc_out(x)

In [8]:

features = df.drop(columns=['thetamaj', 'thetamin','Unnamed: 0', 'ID'])
targets = df[['thetamaj', 'thetamin']]

# Preprocessing
X = MinMaxScaler().fit_transform(features)
y = MinMaxScaler().fit_transform(targets)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

In [ ]:
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch
import numpy as np

# Assume model, loss_fn, device already defined
# Use small subset for testing
X_train_small = X_train[:16]
y_train_small = y_train[:16]

# Define model
model = HybridQNNMulti(input_dim=X_train.shape[1]).to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model with progress bar
epochs = 3  # reduce further if needed

try:
    print("Starting training on small dataset...")
    for epoch in tqdm(range(epochs), desc="Training HQNN"):
        model.train()
        optimizer.zero_grad()
        output = model(X_train_small)
        loss = loss_fn(output, y_train_small)
        loss.backward()
        optimizer.step()
        tqdm.write(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")
except Exception as e:
    print("Error during training:", e)


Starting training on small dataset...


Training HQNN:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Evaluate
model.eval()
with torch.no_grad():
    y_pred = model(X_test).cpu().numpy()
    y_true = y_test.cpu().numpy()

print("\n--- Evaluation Metrics for Both Targets ---")
for i, label in enumerate(['thetamaj', 'thetamin']):
    mse = mean_squared_error(y_true[:, i], y_pred[:, i])
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
    r2 = r2_score(y_true[:, i], y_pred[:, i])
    print(f"{label} - MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")

KeyboardInterrupt: 